# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846926003516                   -0.70    4.5
  2   -7.852331947073       -2.27       -1.53    1.0
  3   -7.852645991074       -3.50       -2.53    2.8
  4   -7.852646676494       -6.16       -3.37    2.5
  5   -7.852646685821       -8.03       -4.76    1.5
  6   -7.852646686722       -9.04       -5.15    4.2
  7   -7.852646686730      -11.15       -5.94    2.0
  8   -7.852646686730      -12.49       -7.39    2.0
  9   -7.852646686730      -14.45       -7.54    2.5
 10   -7.852646686730      -15.05       -8.59    1.5
 11   -7.852646686730   +  -14.57      -10.02    2.5
 12   -7.852646686730      -14.57      -10.34    2.2
 13   -7.852646686730   +  -14.75      -10.86    1.0
 14   -7.852646686730      -14.45       -9.38    1.2
 15   -7.852646686730   +  -14.57       -9.96    1.0
 16   -7.852646686730      -15.05      -10.13    1.0
 17   -7.852646686730   +  -15.05      -11.13 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846791516437                   -0.70           4.5
  2   -7.852527155569       -2.24       -1.64   0.80    2.0
  3   -7.852636416526       -3.96       -2.71   0.80    1.0
  4   -7.852646458550       -5.00       -3.28   0.80    2.2
  5   -7.852646678834       -6.66       -4.08   0.80    1.2
  6   -7.852646686434       -8.12       -4.77   0.80    1.5
  7   -7.852646686722       -9.54       -5.86   0.80    1.5
  8   -7.852646686730      -11.12       -6.75   0.80    2.8
  9   -7.852646686730      -13.08       -7.86   0.80    2.0
 10   -7.852646686730   +  -14.57       -8.11   0.80    2.2
 11   -7.852646686730      -14.57       -8.82   0.80    1.0
 12   -7.852646686730   +  -14.75       -9.50   0.80    1.8
 13   -7.852646686730   +    -Inf      -10.78   0.80    1.8
 14   -7.852646686730   +  -14.75      -10.93   0.80    2.8
 15   -7.852646686730      -14.57      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.384966e+01     4.028776e+00
 * time: 0.5502490997314453
     1     1.187006e+00     1.833639e+00
 * time: 0.7733321189880371
     2    -1.361957e+00     2.074655e+00
 * time: 0.8008129596710205
     3    -3.498887e+00     1.854022e+00
 * time: 0.8400449752807617
     4    -4.686476e+00     1.382134e+00
 * time: 0.8796491622924805
     5    -6.439636e+00     9.479681e-01
 * time: 0.9188289642333984
     6    -7.243313e+00     5.487089e-01
 * time: 0.9576189517974854
     7    -7.569037e+00     3.078092e-01
 * time: 0.9845249652862549
     8    -7.713195e+00     1.197546e-01
 * time: 1.0124800205230713
     9    -7.803358e+00     1.184633e-01
 * time: 1.0400371551513672
    10    -7.834905e+00     6.150196e-02
 * time: 1.0670759677886963
    11    -7.844463e+00     5.414960e-02
 * time: 1.0941040515899658
    12    -7.848733e+00     4.024446e-02
 * time: 1.121124029159546
    13    -7.849194e+00     2.342352e-02
 * time: 1.14834904670

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846826158948                   -0.70    4.8
  2   -7.852287891760       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646685703                   -2.56
  2   -7.852646686730       -8.99       -5.36
  3   -7.852646686730   +  -15.05      -10.03
  4   -7.852646686730   +    -Inf      -16.04


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.752264643159578e-13
|ρ_newton - ρ_scfv| = 1.727735041879446e-13
|ρ_newton - ρ_dm|   = 1.3505596524055624e-9
